### Cookie problem

using CammelCase because of author

In [10]:
from code_.thinkbayes import Pmf

In [17]:
pmf = Pmf()

# priors
pmf.Set('B1', 0.5)
pmf.Set('B2', 0.5)
pmf.GetDict()

{'B1': 0.5, 'B2': 0.5}

In [18]:
# update based on new data (prior * likelihood)
pmf.Mult('B1', 3/4)
pmf.Mult('B2', 1/2)
# After this update, the distribution is no longer normalized, but because these hypotheses
# are mutually exclusive and collectively exhaustive, we can renormalize:
pmf.Normalize()
print(pmf.GetDict())
print(pmf.Prob('B1'))

{'B1': 0.6000000000000001, 'B2': 0.4}
0.6000000000000001


In [26]:
class Cookie(Pmf):
    def __init__(self, hypotheses, mixes=None):
        # calculate prior
        Pmf.__init__(self)
        for H in hypotheses:
            self.Set(H, 1)
        self.Normalize()

        if mixes is None:
            self.mixes = {
                'B1': dict(vanilla=0.75, chocolate=0.25),
                'B2': dict(vanilla=0.5, chocolate=0.5),
            }
        else:
            self.mixes = mixes
    
    def Likelihood(self, data, hypothesis):
        mix = self.mixes[hypothesis]
        likelihood = mix[data]
        return likelihood
    
    def Update(self, data):
        for H in self.Values():
            likelihood = self.Likelihood(data, H)
            self.Mult(H, likelihood)
        self.Normalize()
        return
    

In [27]:
cookies = Cookie(
    ['B1', 'B2']
)
cookies.GetDict()

{'B1': 0.5, 'B2': 0.5}

In [28]:
# we picked vanilla cookie from a bowl
cookies.Update('vanilla')
cookies.GetDict()

{'B1': 0.6000000000000001, 'B2': 0.4}

In [29]:
# we picked chocolate cookie from a bowl
cookies.Update('chocolate')
cookies.GetDict()

{'B1': 0.4285714285714286, 'B2': 0.5714285714285714}

## Monty Hall Problem